In [1]:
import torch
torch.manual_seed(1)

import torch.nn as nn

from torchtext.datasets import IMDB
from torch.utils.data.dataset import random_split

Before we can feed the data into an RNN model, we need to apply several preprocessing steps:
1. Split the training dataset into separate training and validation partitions.
2. Identify the unique words in the training dataset
3. Map each unique word to a unique integer and encode the review text into encoded integers
(an index of each unique word)
4. Divide the dataset into mini-batches as input to the model

In [5]:
train_dataset = IMDB(split='train')
test_dataset = IMDB(split='test')

train_dataset, valid_dataset = random_split(list(train_dataset), [20000, 5000])